In [1]:
import os
import re
import pandas as pd


def load_rock_mds_files(folder):
    pattern = re.compile(r"rocks_mds(\d+)\.txt")
    dfs = []
    for fname in os.listdir(folder):
        match = pattern.match(fname)
        if match:
            subject_number = int(match.group(1))
            df = pd.read_csv(
                os.path.join(folder, fname), delim_whitespace=True, engine="python"
            )
            df["Subject Number"] = subject_number
            dfs.append(df)
    if dfs:
        return pd.concat(dfs, ignore_index=True)
    else:
        return pd.DataFrame()  # Empty if no files found

In [2]:
all_data = load_rock_mds_files(
    "data/Rocks-30 Similarity-Judgment Study/individual_subject_data"
)

C:\Users\crasa\AppData\Local\Temp\ipykernel_30900\2038370909.py:13: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(
C:\Users\crasa\AppData\Local\Temp\ipykernel_30900\2038370909.py:13: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(
C:\Users\crasa\AppData\Local\Temp\ipykernel_30900\2038370909.py:13: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(
C:\Users\crasa\AppData\Local\Temp\ipykernel_30900\2038370909.py:13: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(
C:\Users\crasa\AppData\Local\Temp\ipykernel_30900\2038370909.py:13: FutureWarnin

In [ ]:
all_data["RockPair"] = all_data["RockPair"] = all_data.apply(
    lambda row: tuple(sorted([row["Rock1Type"], row["Rock2Type"]])), axis=1
)
all_data

,Rock1Type,Rock2Type,Rating,Subject Number,RockPair
0,20,18,3,10,"(18, 20)"
1,10,14,7,10,"(10, 14)"
2,4,28,2,10,"(4, 28)"
3,6,24,3,10,"(6, 24)"
4,13,30,6,10,"(13, 30)"
...,...,...,...,...,...
30880,5,22,4,9,"(5, 22)"
30881,4,1,6,9,"(1, 4)"
30882,4,7,5,9,"(4, 7)"
30883,25,14,4,9,"(14, 25)"


In [4]:
len(all_data["Subject Number"].unique())

71

In [7]:
all_data.groupby(["RockPair"])["Rating"].mean().reset_index()

,RockPair,Rating
0,"(1, 2)",5.647887
1,"(1, 3)",6.676056
2,"(1, 4)",6.788732
3,"(1, 5)",5.605634
4,"(1, 6)",3.647887
...,...,...
430,"(27, 29)",2.450704
431,"(27, 30)",3.169014
432,"(28, 29)",1.859155
433,"(28, 30)",2.887324


In [ ]:
df = all_data

results = []

for subject in df["Subject Number"].unique():
    # Data for this subject
    subj_df = df[df["Subject Number"] == subject]
    # Data for all other subjects
    others_df = df[df["Subject Number"] != subject]
    # Mean ratings from others, grouped by Rock1Type and Rock2Type
    mean_ratings = others_df.groupby(["RockPair"])["Rating"].mean().reset_index()
    mean_ratings = mean_ratings.rename(columns={"Rating": "OthersMeanRating"})
    # Merge with this subject's ratings
    merged = pd.merge(subj_df, mean_ratings, on=["RockPair"], how="inner")
    # Compute correlation
    corr = merged["Rating"].corr(merged["OthersMeanRating"])
    results.append({"Subject Number": subject, "Correlation": corr})

result_df = pd.DataFrame(results)
result_df

,Subject Number,Correlation
0,10,0.695675
1,11,0.383672
2,14,0.400213
3,15,0.591911
4,16,0.719669
...,...,...
66,83,0.617614
67,84,0.542280
68,85,0.532101
69,86,0.724805


In [9]:
result_df["Correlation"].mean()

np.float64(0.5790505496883465)

In [10]:
result_df["Correlation"].std()

np.float64(0.1167175441731196)

In [ ]:
import numpy as np

corrs = []
for _ in range(1000):
    subjects = df["Subject Number"].unique()
    np.random.shuffle(subjects)

    # Split into two halves
    half = len(subjects) // 2
    group1 = subjects[:half]
    group2 = subjects[half:]

    # Aggregate average ratings by RockPair for each group
    avg1 = (
        df[df["Subject Number"].isin(group1)]
        .groupby("RockPair")["Rating"]
        .mean()
        .reset_index()
    )
    avg2 = (
        df[df["Subject Number"].isin(group2)]
        .groupby("RockPair")["Rating"]
        .mean()
        .reset_index()
    )

    corr = avg1.merge(avg2, on="RockPair").corr(numeric_only=True).iloc[0, 1]
    corrs.append(float(corr))

In [29]:
np.mean(corrs)

np.float64(0.949111585057384)

In [30]:
np.std(corrs)

np.float64(0.006078413081750423)